Ноутбук для генерации настроечных json файлов для 3-й, 4-й и 5-й версий платформы.

In [2]:
import os
import pandas as pd
import numpy as np
import json 
import math
import string
import re
from transliterate import translit, get_available_language_codes

from sklearn.model_selection import train_test_split

# pd.options.display.max_columns = -1

ModuleNotFoundError: No module named 'transliterate'

In [3]:
?re

In [2]:
RND_SEED = 315

input_dir = 'input'
output_dir = 'output'
faq_dir = os.path.join(output_dir, 'faq')
faq_answers_fname = os.path.join(output_dir, 'faq_answers.json')

train_fname = os.path.join(input_dir, 'train_rnd_lift_100.xlsx')

Чтение исходных данных

In [3]:
train = pd.read_excel(train_fname)
train.head()

,mq_id,master_question,phrase,source
5348,9999,Прочее,добрый день! я могу узнать когда поменялся бик...,rnd
20246,9999,Прочее,в банкомата сегодня снимала деньги там высвети...,rnd
915,1011,Здравствуйте,добрый день!,rnd
8515,1011,Здравствуйте,"здравствуйте, виктория!",rnd
1046,1011,Здравствуйте,добрый день!,rnd


In [4]:
# preprocessing
pattern = re.compile('[\W_]+') 
def remove_punctuations(text):
    for punctuation in string.punctuation:
#         text = text.replace(punctuation, ' ')
        text = pattern.sub(' ', text)

    return text

def normalize(text):
    text = text.lower()
    text = remove_punctuations(text)
    text = text.replace("ё", "е")
    text = text.strip()
    
    return text

In [5]:
train = train.assign(phrase_norm=train['phrase'].apply(lambda t: normalize(t)))

In [6]:
train.drop_duplicates('phrase_norm', inplace=True)

## Train validation split

In [7]:
# train_jsons, train_valid = train_test_split(train, stratify=train['mq_id'].values, test_size=0.5, random_state=RND_SEED)

In [8]:
train_jsons = train.copy()

## Generatig jsons

In [9]:
mq_id_list = train['mq_id'].unique()
mq_id_other_list = []

In [10]:
#Ветка release 004 & 005
def generate_jsons(phrases_df, 
                   mq_id_colname, 
                   mq_colname,
                   phrase_colname, 
                   mq_id_list, 
                   mq_id_other_list,
                   answers_fname, 
                   faq_dir):
    
    CHANNELS_LIST = [
          "SBERBANK_MARKETS",
          "SBERBANK_MESSENGER",
          "TELEGRAM"
        ]
    
    res = []
    faq_answers = {'action': {"command": "answer_to_user"}, }

    for mq_id in list(mq_id_list):
        local_obj = {}
        df_mq_id = phrases_df[phrases_df[mq_id_colname] == mq_id]
        samples_positive = df_mq_id[phrase_colname].unique()
        mq = str(df_mq_id[mq_colname].iloc[0])
        
        if len(samples_positive) > 0:
            if mq_id in mq_id_other_list:
                local_obj['other'] = True

            local_obj['channels'] = CHANNELS_LIST
            local_obj['samples_positive'] = list(samples_positive)

            local_obj['tags'] = []
            local_obj['id'] = int(mq_id)
            local_obj['version'] = 1
            name = translit(mq.replace('/', '_').replace(' ', '_'), 'ru', reversed=True)
            local_obj['name'] = name

    #         if settings_scales[settings_scales['mq_id'] == mq_id].shape[0] > 0:
    #             if settings_scales.loc[mq_id]['scale'] != 1:
    #                 local_obj['distance_scale_factor'] = settings_scales.loc[mq_id]['scale']

            file_name = name + '.json'    

            with open(os.path.join(faq_dir, file_name), 'wt', encoding='utf8') as outfile:
                json.dump([local_obj], outfile, indent=4, ensure_ascii=False)
            
            answer_str = 'answer {}'.format(mq_id)
            faq_answers[str(local_obj['id'])] = \
                {"action": {"command": "answer_to_user",\
                            "nodes": { "answer": [answer_str]}},\
                "master_question_action": {"type": "do_nothing",\
                                           "command": "answer_to_user",\
                                           "nodes": {"answer": mq}}}



    with open(answers_fname, 'wt', encoding='utf8') as outfile:
        json.dump(faq_answers, outfile, indent=4, ensure_ascii=False)

In [11]:
generate_jsons(phrases_df=train_jsons, 
                   mq_id_colname='mq_id', 
                   mq_colname='master_question',
                   phrase_colname='phrase', 
                   mq_id_list=mq_id_list, 
                   mq_id_other_list=mq_id_other_list,
                   answers_fname=faq_answers_fname, 
                   faq_dir=faq_dir)